In [12]:
import numpy as np
import pandas as pd
import bs4
import folium
import requests
# import geopy
import urllib3

Grabbing URL with urllib and scraping the html with Beautiful Soup

In [13]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
urlLib = urllib3.PoolManager()
r = urlLib.request("GET", url)

soup = bs4.BeautifulSoup(r.data)

/Users/isaactrussell/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/isaactrussell/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/isaactrussell/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Running through the first table and turning it into a pandas dataframe

In [14]:
table = soup.findChildren("table")[0]
rows = []
for row in table.find_all("tr"):
        temp = []
        elems = row.find_all("td")
        for elem in elems:
            if elems[-1] == elem:
                text = elem.text
                text = text[:-1]
                temp.append(text)
            else:
                temp.append(elem.text)
        rows.append(temp)
    
df = pd.DataFrame.from_records(rows[1:])
headers = [head.text for head in table.find_all("th")]
headers[-1] = headers[-1][:-1]
df.columns = headers

print(df.shape)
df.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping all the "Not assigned" boroughs and assign the borough names for neighbourhoods that lack a name

In [15]:
mask = df["Borough"] != "Not assigned"
df = df[mask]
print(df.shape)
df.head()

row = df[df["Neighbourhood"] == "Not assigned"]
df.at[row.index, "Neighbourhood"] = row.Borough.values[0]

(211, 3)


Gather all the unique postal code values

In [16]:
code_list = df["Postcode"].unique()
print(code_list)

['M3A' 'M4A' 'M5A' 'M6A' 'M7A' 'M9A' 'M1B' 'M3B' 'M4B' 'M5B' 'M6B' 'M9B'
 'M1C' 'M3C' 'M4C' 'M5C' 'M6C' 'M9C' 'M1E' 'M4E' 'M5E' 'M6E' 'M1G' 'M4G'
 'M5G' 'M6G' 'M1H' 'M2H' 'M3H' 'M4H' 'M5H' 'M6H' 'M1J' 'M2J' 'M3J' 'M4J'
 'M5J' 'M6J' 'M1K' 'M2K' 'M3K' 'M4K' 'M5K' 'M6K' 'M1L' 'M2L' 'M3L' 'M4L'
 'M5L' 'M6L' 'M9L' 'M1M' 'M2M' 'M3M' 'M4M' 'M5M' 'M6M' 'M9M' 'M1N' 'M2N'
 'M3N' 'M4N' 'M5N' 'M6N' 'M9N' 'M1P' 'M2P' 'M4P' 'M5P' 'M6P' 'M9P' 'M1R'
 'M2R' 'M4R' 'M5R' 'M6R' 'M7R' 'M9R' 'M1S' 'M4S' 'M5S' 'M6S' 'M1T' 'M4T'
 'M5T' 'M1V' 'M4V' 'M5V' 'M8V' 'M9V' 'M1W' 'M4W' 'M5W' 'M8W' 'M9W' 'M1X'
 'M4X' 'M5X' 'M8X' 'M4Y' 'M7Y' 'M8Y' 'M8Z']


Drop all duplicates and assign new neighbourhood column with the join neighbourhood data 

In [17]:
df_dropped = df.drop_duplicates(["Postcode"], keep = "first")
df_dropped.drop("Neighbourhood", axis = 1, inplace = True)
print("Number of Unique Postal Codes: {}".format(df_dropped.shape[0]))

neighbourhoods = []

for code in code_list:
    hoods = df["Neighbourhood"][df["Postcode"] == code].values
    neighbourhoods.append(",".join(hoods))
df_dropped["Neighbourhood"] = neighbourhoods
df_dropped.reset_index(drop = True, inplace = True)
df_dropped.head()

Number of Unique Postal Codes: 103


/Users/isaactrussell/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/isaactrussell/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Printing shape of the dataset

In [18]:
df = df_dropped
print(df.shape)

# # df.drop(["Latitude", "Longitude"], axis = 1, inplace = True)
# df.head()

(103, 3)


In [19]:
# Tried to run geocoder, loop ran for 30 seconds with no output on a single postal code. Decided to simply use the csv provided

# import geocoder

# coords = None

# while coords is None:
#     g = geocoder.google("M2K, Toronto, Ontario")
#     coords = g.latlng
# print(coords)

Using join method to create a new dataframe of the existing data along with the latitude and longitude data.

In [20]:
latLon = pd.read_csv("Geospatial_Coordinates.csv")
latLon.head()

latLon.sort_values(by = "Postal Code", inplace = True)
df.sort_values(by = "Postcode", inplace = True)

df = df.join(latLon.set_index("Postal Code"), on = "Postcode")
df.reset_index(drop = True, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Practice requesting data from the Foursquare API using the first location in DataFrame

In [22]:
import requests
import json

ll = df.iloc[0][["Latitude", "Longitude"]].values
value = ",".join(str(val) for val in ll)

url = "https://api.foursquare.com/v2/venues/explore"
params = dict(
    client_id = 'W5CHCMS4RL2BMAQVYOORLJQCF4XLGWK42SDQWAG1XFLX2LRV',
    client_secret = 'BVUEP1F3ZHWSQGN24XBSDK0GK15CBU2OJZ53VLQIR4PQQQ4U',
    v = '20180323',
    ll = value,
    limit = 10,
    radius = 600)

# print(params)
req = requests.get(url = url, params = params)
data = json.loads(req.text)

Showing all data in response section of json file with their respective keys

In [23]:
keys = data["response"].keys()

for key in keys:
    print("--" + key + "--")
    print(data["response"][key], "\n")

--headerLocation--
Malvern 

--headerFullLocation--
Malvern, Toronto 

--headerLocationGranularity--
neighborhood 

--totalResults--
4 

--suggestedBounds--
{'ne': {'lat': 43.8120863054, 'lng': -79.1868848168765}, 'sw': {'lat': 43.80128629459999, 'lng': -79.20182198312352}} 

--groups--
[{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4d669cba83865481c948fa53', 'name': 'Images Salon & Spa', 'contact': {}, 'location': {'address': '8130 Sheppard Ave E', 'crossStreet': 'Morningside Ave', 'lat': 43.80228301948931, 'lng': -79.19856472801668, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80228301948931, 'lng': -79.19856472801668}], 'distance': 595, 'postalCode': 'M1B 3W3', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)', 'Toronto ON M1B 3W3', 'C



The section "group" seems to have the majority of the valuable information contained in it. 

The total results section would be an interesting thing to explore as it can be an indicator of density in the given radius.

Further investigating the "groups" section to understand json file structure

In [24]:

results = data["response"]["groups"][0]["items"]
venue_keys = list(results[0]['venue'].keys())
for key in venue_keys:
    print("--{}--".format(key))
    print(results[0]["venue"][key], "\n")

--id--
4d669cba83865481c948fa53 

--name--
Images Salon & Spa 

--contact--
{} 

--location--
{'address': '8130 Sheppard Ave E', 'crossStreet': 'Morningside Ave', 'lat': 43.80228301948931, 'lng': -79.19856472801668, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80228301948931, 'lng': -79.19856472801668}], 'distance': 595, 'postalCode': 'M1B 3W3', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)', 'Toronto ON M1B 3W3', 'Canada']} 

--categories--
[{'id': '4bf58dd8d48988d1ed941735', 'name': 'Spa', 'pluralName': 'Spas', 'shortName': 'Spa', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_', 'suffix': '.png'}, 'primary': True}] 

--verified--
False 

--stats--
{'tipCount': 0, 'usersCount': 0, 'checkinsCount': 0, 'visitsCount': 0} 

--beenHere--
{'count': 0, 'lastCheckinExpiredAt': 0, 'marked': False, 'unconfirmedCount': 0} 

--photos--
{'count': 0, 'groups': []} 

--hereNow--
{'count': 0, 

Simlar to the New York city example. Grabbing the types of places in the area will be a good feature to draw on. Another feature could be the amount of check ins or visits that a venue has. Could be an indicator of community engagment. Let's check out some other neighborhoods and see how our results differ from this one. There were only 4 results returned here which seems like a small amount.

First lets make it easier to grab and format the lat and long values from our DataFrame with a quick function and a function to make requests easier

In [25]:
### Takes in a row number, i,  and returns the corresponding lat and long values
def get_lat_long(i):
    ll = df.iloc[i][["Latitude", "Longitude"]].values
    return ",".join(str(val) for val in ll)

### Lets also make a quick way to request from the Foursquare API
#Input : value - row number we want to learn about
#        rad (optional: default = .5 mile) - radius around search point
#        limit (optional: default = 15) - number of results to output from the API
#Output:
#       num_results - number of results from request
#       data - "important" data from the groups section of the json file

def makeRequest(value, rad = 805, limit = 15):
    ll = get_lat_long(value)
    url = "https://api.foursquare.com/v2/venues/explore"
    params = dict(
        client_id = 'W5CHCMS4RL2BMAQVYOORLJQCF4XLGWK42SDQWAG1XFLX2LRV',
        client_secret = 'BVUEP1F3ZHWSQGN24XBSDK0GK15CBU2OJZ53VLQIR4PQQQ4U',
        v = '20180323',
        ll = ll,
        limit = limit,
        radius = rad)
    req = requests.get(url = url, params = params)
    data = json.loads(req.text)["response"]
    return data["totalResults"], data["groups"]


Lets test it out on a couple more rows

In [ ]:
# Random numbers
test_rows = [10, 36, 58, 86, 99, 101]
for row in test_rows:
    n, new_data = makeRequest(row)
    print("Number of Results from Request: {}".format(n))

There seems to be a good variety of result sizes. Now lets check out all the neighborhood location on a map

In [ ]:
toronto_lat_long = [43.7, -79.4]
m = folium.Map(location = toronto_lat_long, zoom_start = 11, tiles = "Stamen Toner")

for row in range(df.shape[0]):
    folium.Circle(location = df.iloc[row][["Latitude", "Longitude"]].values,
                  color = "crimson", 
                  popup = df.iloc[row]["Neighbourhood"],
                  fill = True,
                  radius = 90).add_to(m)
m